# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
#    dependencies and setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from datetime import datetime

#    import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
#   load the "cities" csv file created in "weatherpy" to a dataframe and check its contents

#   configure google maps to use my API key
gmaps.configure(api_key=g_key)

#   load the csv
city_weather_df = pd.read_csv("cities.csv", encoding="utf-8")

#   check that "cities.csv" returns 570 rows of found cities as expected
city_weather_df


,City,Country,Max Temperature,Humidity,Cloudiness,Windspeed,Latitude,Longitude,Date
0,port alfred,ZA,72.64,92,51,14.72,-33.5906,26.8910,1641768476
1,mar del plata,AR,78.01,36,0,8.99,-38.0023,-57.5575,1641768671
2,kwekwe,ZW,65.73,92,100,2.44,-18.9281,29.8149,1641768424
3,malente,DE,39.11,97,100,5.19,54.1712,10.5552,1641768672
4,punta arenas,CL,50.11,66,75,17.27,-53.1500,-70.9167,1641768673
...,...,...,...,...,...,...,...,...,...
565,chimoio,MZ,71.46,88,75,3.44,-19.1164,33.4833,1641769032
566,khorramshahr,IR,51.76,93,57,4.61,30.4397,48.1664,1641769033
567,saint anthony,US,5.94,58,40,4.00,45.0205,-93.2180,1641769034
568,alyangula,AU,84.07,70,54,4.61,-13.8483,136.4192,1641769034


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
#   format the date in the weather_df dataset as "run_date"
run_date = city_weather_df["Date"].max()
rundate = datetime.utcfromtimestamp(run_date).strftime('%m-%d-%Y')

In [4]:
#   Use the Latitude and Longitude as locations.
locations = city_weather_df[["Latitude","Longitude"]]

#   Add the heat map layer to the map.
humidity = city_weather_df["Humidity"]
 
#   Plot plot heat map
fig = gmaps.figure(center=(46.0, -5.0), zoom_level=2)

#   Add the heat layer
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=np.max(humidity), point_radius=3)

#   Add the heat layer to the figure and show the figure 
fig.add_layer(heat_layer)

#   save the fig 
#savefig("Humidity Heatmap on {rundate}.png")

fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
#   narrow down the cities to fit required weather conditions dropping Null values and displaying the dataframe.
ideal_city_weather_df = city_weather_df.loc[(city_weather_df["Windspeed"] <= 10) & (city_weather_df["Cloudiness"]== 0) & \
                  (city_weather_df["Max Temperature"] >= 70) & (city_weather_df["Max Temperature"] <= 80)].dropna()
ideal_city_weather_df


,City,Country,Max Temperature,Humidity,Cloudiness,Windspeed,Latitude,Longitude,Date
1,mar del plata,AR,78.01,36,0,8.99,-38.0023,-57.5575,1641768671
71,cape town,ZA,71.10,76,0,1.01,-33.9258,18.4232,1641768712
108,lompoc,US,70.79,67,0,8.05,34.6391,-120.4579,1641768733
418,marana,US,73.94,29,0,5.75,32.4367,-111.2254,1641768935
543,julio de castilhos,BR,77.38,41,0,8.72,-29.2269,-53.6817,1641769018


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
#   Create the "hotel_df" dataframe.
hotel_df = ideal_city_weather_df.loc[:,["City","Country","Latitude","Longitude"]]

#   Add the "Hotel Name" column to the dataFrame and display the dataframe.
hotel_df["Hotel Name"] = ""
hotel_df


,City,Country,Latitude,Longitude,Hotel Name
1,mar del plata,AR,-38.0023,-57.5575,
71,cape town,ZA,-33.9258,18.4232,
108,lompoc,US,34.6391,-120.4579,
418,marana,US,32.4367,-111.2254,
543,julio de castilhos,BR,-29.2269,-53.6817,


In [7]:
#Set parameters to search for hotels with 5000 meters.
#Hit the Google Places API for each city's coordinates.
#Store the first Hotel result into the DataFrame.
#Plot markers on top of the heatmap.

In [8]:
#   Set the parameters to search for hotels within 5000 meters.
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"
params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}


In [9]:
#   Hit the Google Places API for each city's coordinates.

#   counter for delay after every 50 cities to avoid API errors due to large calls
counter = 0

#   counters for cities found and cities not found
found_counter = 0
not_found_counter = 0

#   store the first Hotel result into the DataFrame
for index, row in hotel_df.iterrows():
    try:
        latitude = row["Latitude"]
        longitude = row["Longitude"]
        city_name = row["City"]

        params["location"] = f"{latitude},{longitude}"

        print(f"Retrieving Results for Index {index}: {city_name}.")
        response = requests.get(base_url, params=params).json()
        results = response['results']

#   Save the hotel name to the dataframe.   
        counter = counter + 1
        found_counter = found_counter + 1
        #print(f"City No.: {found_counter}, "f"Found: {city}")

        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    except (KeyError, IndexError):
        counter = counter + 1
        not_found_counter = not_found_counter + 1
        print(f"City No.: {not_found_counter}", f"NOT Found: {city_name}")

#   uncomment the sleep code if errors occur with this large API call
        #if counter == 50: 
            #time.sleep(60)
            #counter = 0
            
#   debugging code to limit search if required
    #if counter == 150: 
        #break 

print("*** Search has ended ***")
hotel_df
        

Retrieving Results for Index 1: mar del plata.
Closest hotel in mar del plata is NH Gran Hotel Provincial.
Retrieving Results for Index 71: cape town.
Closest hotel in cape town is The Bay Hotel.
Retrieving Results for Index 108: lompoc.
Closest hotel in lompoc is Hilton Garden Inn Lompoc.
Retrieving Results for Index 418: marana.
City No.: 1 NOT Found: marana
Retrieving Results for Index 543: julio de castilhos.
Closest hotel in julio de castilhos is Pousada Diana.
*** Search has ended ***


,City,Country,Latitude,Longitude,Hotel Name
1,mar del plata,AR,-38.0023,-57.5575,NH Gran Hotel Provincial
71,cape town,ZA,-33.9258,18.4232,The Bay Hotel
108,lompoc,US,34.6391,-120.4579,Hilton Garden Inn Lompoc
418,marana,US,32.4367,-111.2254,
543,julio de castilhos,BR,-29.2269,-53.6817,Pousada Diana


In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [13]:
# Add marker layer ontop of heat map
hotel_markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(hotel_markers)

#   save the fig of the the map
#fig.savefig('TargetHotels.png')

#   despite several attempts at AskBCS to save the figure with code I resorted to downloading it manually to get it done.

# Display figure
fig


Figure(layout=FigureLayout(height='420px'))

In [12]:
#   the map shows the 5 hotels that satisfied the ideal weather criteria 
